In [ ]:
## This notebook makes predictions over test images using trained predictor

In [ ]:
# import packages
import os, cv2
import numpy as np
import pandas as pd
from datetime import datetime
## packages for deep learning
import tensorflow as tf
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from tensorflow.python.client import device_lib
## check GPU
print(device_lib.list_local_devices())

In [ ]:
# load the pretrained model
THRESH = 20
n_c = 25
path_wd = '../' ## set working directory
if not os.path.isdir(path_wd + 'output/npy/npy_' + str(THRESH) + '_' + str(n_c)):
    os.mkdir(path_wd + 'output/npy/npy_' + str(THRESH) + '_' + str(n_c))
time_start = datetime.now()
print('Start:', time_start)
model_resnet = tf.keras.models.load_model(path_wd + 'output/models/APL_' + str(THRESH) + '_' + str(n_c))
print('Finished:', datetime.now() - time_start)

In [ ]:
# loop over the shadow_free folder
folder_name = path_wd + 'data/test/'
file_list = os.listdir(folder_name)
rs = 10
size_wind = 100
num_col = int(10000/rs)
for img_name in file_list:
    print(img_name)
    probs_mat = np.zeros((num_col, num_col, 3))
    img_path = folder_name + img_name
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) ## transform color channel
    x_img = np.ones((num_col, 100, 100, 3), dtype=np.uint8)
    ## start prediction
    time_start = datetime.now()
    ## use the pretrained model as convolution kernel
    for i in range(num_col):
        r_1 = int(np.max([0, i*rs + rs/2 - size_wind/2]))
        r_2 = int(np.min([10000, i*rs + rs/2 +  size_wind/2]))
        for j in range(num_col):
            c_1 = int(np.max([0, j*rs + rs/2 - size_wind/2]))
            c_2 = int(np.min([10000, j*rs + rs/2 + size_wind/2]))
            img_patch = img[r_1:r_2, c_1:c_2]
            if img_patch.shape != (100, 100, 3):
                x_img[j] = cv2.resize(img_patch, (100, 100)) ## boundary and corner patches
            else:
                x_img[j] = img_patch
        x_img = preprocess_input(x_img)
        probs_mat[i] = model_resnet.predict(x_img)
    npy_path = path_wd + 'output/npy/npy_' + str(THRESH) + '_' + str(n_c) + '/' + img_name[:(-4)] + '.npy'
    np.save(npy_path, probs_mat)
    print('Time:', datetime.now() - time_start)
print('All finished!')